In [1]:
from nemo.collections.asr.models import EncDecSpeakerLabelModel
from IPython.display import Audio, display
import numpy as np
import torch
import torchaudio
import torchaudio.transforms as T
import torchaudio.functional as F
import math
import os
import requests
import boto3
import wave
import sys
import contextlib
import webrtcvad
import collections
from tqdm import tqdm
from tqdm.notebook import tqdm
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score, davies_bouldin_score
from sklearn.decomposition import PCA
from sklearn.mixture import GaussianMixture
from itertools import combinations


[NeMo W 2023-06-28 13:04:02 optimizers:54] Apex was not found. Using the lamb or fused_adam optimizer will error out.
[NeMo W 2023-06-28 13:04:07 nemo_logging:349] /Users/ajaybati/miniconda3/envs/sanas/lib/python3.8/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
      warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)
    


In [2]:
spEmbs = torch.load("SpeakerNetEmbs.pkl")
ecapaEmbs = torch.load("EcapaEmbs.pkl")

In [399]:
pos1000 = np.array([np.random.choice(len(x),7, replace=False) for x in spEmbs])
pos1000 = [list(combinations(x, 2)) for x in pos1000]

In [400]:
negCands = [np.random.choice(54,len(pos1000[0]),replace=True) for x in range(54)]
neg1000 = [[(2,x,np.random.choice(len(spEmbs[x]))) for x in negCands[y]] for y in range(len(spEmbs))]

In [401]:
print(len(pos1000)*len(pos1000[0]))

2430


In [402]:
#len(pos/neg1000) = 54, each index is for each speaker
def get_score(embs, cand1, cand2):
    #cand1/2 = (ith speaker, jth segment)
    X = embs[cand1[0]][cand1[1]]
    Y = embs[cand2[0]][cand2[1]]
    # Length Normalize
    # X = embs1 / torch.linalg.norm(embs1)
    # Y = embs2 / torch.linalg.norm(embs2)
    # Score
    similarity_score = torch.dot(X, Y) / ((torch.dot(X, X) * torch.dot(Y, Y)))
    # similarity_score = (similarity_score + 1) / 2
    return similarity_score.item()

In [403]:
trueLabelsSP = []
scoresSP = []
for x in range(len(pos1000)):
    for ind1, ind2 in pos1000[x]:
        scoresSP.append(get_score(spEmbs, (x, ind1), (x, ind2)))
        trueLabelsSP.append(1)
for x in range(len(neg1000)):
    for indCur, ind1, ind2 in neg1000[x]:
        scoresSP.append(get_score(spEmbs, (x, indCur), (ind1, ind2)))
        trueLabelsSP.append(0)

In [404]:
trueLabelsEC = []
scoresEC = []
for x in range(len(pos1000)):
    for ind1, ind2 in pos1000[x]:
        scoresEC.append(get_score(ecapaEmbs, (x, ind1), (x, ind2)))
        trueLabelsEC.append(1)
for x in range(len(neg1000)):
    for indCur, ind1, ind2 in neg1000[x]:
        scoresEC.append(get_score(ecapaEmbs, (x, indCur), (ind1, ind2)))
        trueLabelsEC.append(0)

In [405]:
from sklearn.metrics import roc_curve
def compute_eer(distances, labels):
    fprs, tprs, _ = roc_curve(labels, distances)
    eer = fprs[np.argmin(np.absolute((1 - tprs) - fprs))] #get the FPR where FPR ~= FNR (== 1-TPR)
    # print(np.min(np.absolute((1 - tprs) - fprs)))
    return eer

In [406]:
compute_eer(scoresSP, trueLabelsSP)

0.2876543209876543

In [407]:
compute_eer(scoresEC, trueLabelsEC)

0.21522633744855968